### Ejemplos de PySpark ML (Machine Learning)

#### INICIAMOS SESIÓN DE SPARK

In [ ]:
# Importamos SparkSession, que es el punto de entrada para trabajar con Spark
from pyspark.sql import SparkSession

# Creamos una sesión de Spark con el nombre 'dataframe'
# getOrCreate() crea una nueva sesión o devuelve una existente si ya hay una
spark = SparkSession.builder.appName('dataframe').getOrCreate()

In [ ]:
# Listamos los archivos disponibles en el directorio de FileStore de Databricks
# Esta es una ubicación donde puedes subir tus propios archivos
display(dbutils.fs.ls("dbfs:/FileStore/tables/"))

In [ ]:
# Cargamos el archivo CSV de entrenamiento
# inferSchema=True permite a Spark detectar automáticamente los tipos de datos
# header=True indica que la primera fila contiene los nombres de las columnas
training = spark.read.csv("dbfs:/FileStore/tables/test1.csv", inferSchema=True, header=True)

In [ ]:
# Mostramos información sobre la sesión de Spark
# Esto nos da detalles sobre la configuración y el estado de la sesión
spark

CARGAMOS EL ARCHIVO CSV

training = spark.read.csv('data/test1.csv', header=True, inferSchema=True)

In [ ]:
# Cargamos los datos de entrenamiento desde un archivo CSV local
# Este es un método alternativo cuando trabajas en un entorno local (no Databricks)
training = spark.read.csv('data/test1.csv', header=True, inferSchema=True)

In [ ]:
# Mostramos el contenido del DataFrame de entrenamiento
# Podemos ver que tenemos datos sobre nombres, edad, experiencia y salario
training.show()

En PySpark se trabaja de forma diferente a otros frameworks de ML. Tendremos que agrupar nuestras variables independientes de forma que queden todas en una columna y dentro de un vector, por lo que crearemos un **vector assembler** (ensamblador de vectores), de tal modo que queden así esas variables independientes:
- [Age, Experience]

Lo que haremos con estas dos variables, será tratarlas como una nueva variable independiente:
- [Age, Experience] ----> nueva_variable_independiente

Esto es un requisito específico de PySpark ML para el procesamiento de características.

In [ ]:
# Importamos VectorAssembler, que es una herramienta de transformación de características
# Esta clase nos permite combinar múltiples columnas en un único vector de características
from pyspark.ml.feature import VectorAssembler

In [ ]:
# Creamos un VectorAssembler que combinará nuestras variables independientes
# inputCols: lista de columnas que queremos usar como características (age y Experience)
# outputCol: nombre de la nueva columna que contendrá el vector de características
feature_assembler = VectorAssembler(inputCols=['age', 'Experience'], outputCol='Independent features')

In [ ]:
# Aplicamos la transformación al DataFrame de entrenamiento
# transform() crea una nueva columna con el vector de características combinadas
output = feature_assembler.transform(training)

Veremos que se crea una nueva columna cuyos valores se corresponden a vectores (arrays) con el contenido de aquellas variables independientes que hemos agrupado. Esta columna será nuestro **input feature** o lo que solíamos definir como X (variables predictoras) en otros frameworks de Machine Learning.

In [ ]:
# Mostramos el DataFrame transformado
# Observa la nueva columna "Independent features" que contiene vectores [age, Experience]
output.show()

Seleccionamos las columnas que nos interesan para nuestro modelo: 
- **Independent features**: nuestras variables predictoras (X)
- **Salary**: nuestra variable objetivo (y)

In [ ]:
# Seleccionamos solo las columnas relevantes para el modelo de ML
# Eliminamos columnas como "Name" que no aportan valor predictivo
finalized_data = output.select('Independent features', 'Salary')

# Mostramos el DataFrame final listo para entrenar el modelo
finalized_data.show()

A continuación, entrenaremos un **modelo de regresión lineal** para predecir el salario basándonos en la edad y la experiencia.

In [ ]:
# Importamos el algoritmo de regresión lineal desde el módulo de ML de PySpark
from pyspark.ml.regression import LinearRegression

In [ ]:
# Dividimos los datos en conjuntos de entrenamiento (75%) y prueba (25%)
# randomSplit([0.75, 0.25]) divide aleatoriamente el DataFrame en dos partes
(train, test) = finalized_data.randomSplit([0.75, 0.25])

# Creamos el modelo de regresión lineal
# featuresCol: columna que contiene las variables predictoras (X)
# labelCol: columna que contiene la variable objetivo (y)
regressor = LinearRegression(featuresCol='Independent features', labelCol='Salary')

# Entrenamos el modelo con los datos de entrenamiento
# fit() ajusta el modelo a los datos
regressor = regressor.fit(train)

In [ ]:
# Obtenemos los coeficientes del modelo entrenado
# Estos representan el peso de cada variable independiente en la predicción
# En regresión lineal: y = β₀ + β₁*x₁ + β₂*x₂
# coefficients contiene [β₁, β₂] (los pesos para age y Experience)
regressor.coefficients

In [ ]:
# Obtenemos el intercepto (término independiente) del modelo
# En regresión lineal: y = β₀ + β₁*x₁ + β₂*x₂
# intercept es β₀ (el valor base cuando todas las variables son 0)
regressor.intercept

In [ ]:
# Evaluamos el modelo con el conjunto de prueba
# evaluate() genera predicciones y calcula métricas de rendimiento
prediction = regressor.evaluate(test)

In [ ]:
# Mostramos las predicciones del modelo
# Compara los valores reales (Salary) con los valores predichos (prediction)
# Esto nos permite ver qué tan bien está prediciendo el modelo
prediction.predictions.show()

In [ ]:
# Calculamos las métricas de error del modelo

# MAE (Mean Absolute Error) - Error Absoluto Medio
# Mide la diferencia promedio absoluta entre predicciones y valores reales
# Valores más bajos indican mejor rendimiento

# MSE (Mean Squared Error) - Error Cuadrático Medio
# Penaliza más los errores grandes al elevarlos al cuadrado
# Valores más bajos indican mejor rendimiento
prediction.meanAbsoluteError, prediction.meanSquaredError